<a href="https://colab.research.google.com/github/ydchen17/RugPullDetection/blob/main/Do_not_rug_on_me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep

In [1]:
!pip install --force-reinstall jsonschema==3.2.0 web3 pycryptodome

     |████████████████████████████████| 56 kB 2.7 MB/s 
     |████████████████████████████████| 499 kB 9.8 MB/s 
     |████████████████████████████████| 2.0 MB 47.8 MB/s 
     |████████████████████████████████| 60 kB 7.7 MB/s 
     |████████████████████████████████| 117 kB 52.8 MB/s 
     |████████████████████████████████| 1.1 MB 54.0 MB/s 
     |████████████████████████████████| 101 kB 5.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 1.1 MB 46.0 MB/s 
     |████████████████████████████████| 103 kB 13.7 MB/s 
     |████████████████████████████████| 1.1 MB 43.5 MB/s 
     |████████████████████████████████| 82 kB 333 kB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 37.3 MB/s 
     |████████████████████████████████| 271 kB 73.6 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 48 kB 5.2 MB/s 
     |████████████████████████

In [3]:
!pip install eth-tester

     |████████████████████████████████| 69 kB 3.3 MB/s 


In [6]:
!pip install --force-reinstall typing_extensions

  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [ ]:
!git clone https://github.com/T2Project/RugPullDetection.git

Cloning into 'RugPullDetection'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 146 (delta 31), reused 49 (delta 16), pack-reused 58
Receiving objects: 100% (146/146), 18.95 MiB | 16.96 MiB/s, done.
Resolving deltas: 100% (42/42), done.


# Shared

In [7]:
import configparser
import subprocess
from web3 import Web3

class shared:
  def init():
    global INFURA_URL
    global USE_POOL_INFURA
    global USE_WALLET_INFURA
    global API_KEY
    global UNISWAP_FACTORY
    global SUSHISWAP_FACTORY
    global UNILOCK_ADDRESS
    global EXCHANGES
    global ETH_ADDRESS
    global DEAD_ADDRESS
    global WETH
    global TOKEN_TOLERANCE
    global POOLS
    global FAKE_POOL
    global MULTICALL_CONTRACT
    global TRANSACTION
    
    global ROOT_FOLDER
    global WEEK
    global ABI
    global ABI_FACTORY
    global BLOCKSTUDY
    global ABI_POOL
    
    global web3
    global multicall

    ROOT_FOLDER        = subprocess.run(["git", "rev-parse", "--show-toplevel"], check=True, universal_newlines=True, stdout=subprocess.PIPE).stdout.strip()
    config = configparser.ConfigParser()
    config.read(ROOT_FOLDER + "/config.ini")

    # # Get infura node url and logic booleans
    INFURA_URL         = config.get("NODE", "INFURA_URL")
    USE_POOL_INFURA    = config.getboolean("NODE", "USE_POOL_INFURA")
    USE_WALLET_INFURA  = config.getboolean("NODE", "USE_WALLET_INFURA")
    
    # Get etherscan keys
    API_KEY            = config.get("APIS", "ETHERSCAN_API_KEY")

    # Get tokens addresses
    ETH_ADDRESS        = config.get("ADDRESS", "ETH_ADDRESS")
    DEAD_ADDRESS       = config.get("ADDRESS", "DEAD_ADDRESS")
    WETH               = config.get("ADDRESS", "WETH_ADDRESS")
    MULTICALL_CONTRACT = config.get("ADDRESS", "MULTICALL_CONTRACT")

    # Get routers addresses
    UNISWAP_ADDRESS    = config.get("ROUTERS", "UNISWAP_ADDRESS")
    UNILOCK_ADDRESS    = config.get("ADDRESS", "UNILOCK_ADDRESS")
    SUSHISWAP_ADDRESS  = config.get("ROUTERS", "SUSHISWAP_ADDRESS")
    FAKE_POOL          = config.get("POOLS", "FAKE_POOL")

    # Get pools factories
    UNISWAP_FACTORY    = config.get("FACTORIES", "UNISWAP_FACTORY")

    #Get log hashes
    TRANSACTION        = config.get("LOG_HASHES","TRANSACTION")

    EXCHANGES          = {
        "uniswap": UNISWAP_ADDRESS.lower(),
        "sushiswap": SUSHISWAP_ADDRESS.lower()
    }
    
    TOKEN_TOLERANCE    = float(config.get("THRESHOLDS", "TOKEN_TOLERANCE"))
    # Define other globals not saved in config.ini
    WEEK               = 4*60*24*7
    ABI                = open(ROOT_FOLDER + "/ABIs/normal_token_abi.txt").read()
    ABI_FACTORY        = open(ROOT_FOLDER + "/ABIs/factory_abi.txt").read()
    ABI_POOL           = open(ROOT_FOLDER + "/ABIs/abi_pool.txt").read()
    # Define global objects
    BLOCKSTUDY        =  13152303

    web3 = Web3(Web3.HTTPProvider(INFURA_URL))
    # multicall =  Multicall(web3.eth)

FileNotFoundError: ignored

# Utility

In [ ]:
from Crypto.Hash import keccak


def obtain_hash_event(event: str) -> str:
    k = keccak.new(digest_bits=256)
    k.update(bytes(event, encoding='utf-8'))
    return '0x'+k.hexdigest()


def obtain_events_from_abi(abi: dict) -> list:
    events = []
    for function in abi:
        if function['type'] == 'event':
            event = function['name'] + '('
            for cont, element in enumerate(function['inputs']):
                if cont == 0:
                    event += element['type']

                else:
                    event += ','+element['type']

                cont += 1

            event  += ')'
            events.append(event)

    return events

In [ ]:
import requests
from web3.datastructures import AttributeDict
from hexbytes import HexBytes
import sys
sys.path.append("../")
shared.init()


def get_rpc_response(method, list_params=[]):
    """
    Parameters
    ----------
    method: str
        Indicates node method.
    list_params: List[Dict[str, Any]]
        List of request parameters.
    Returns
    -------
    args_event: AttributeDict
        Change number basis.
    Example
    -------
        If we want token transfers of 0xa150Db9b1Fa65b44799d4dD949D922c0a33Ee606
        between blocks [11000000, 11025824] then:
        method: 'eth_getLogs'
        list_params: [[{'address': '0xa150Db9b1Fa65b44799d4dD949D922c0a33Ee606',
                    'fromBlock': '0xa7d8c0', 'toBlock': '0xa83da0',
                    'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef']}]]
    """
    url = shared.INFURA_URL
    list_params = list_params or []
    data = [{"jsonrpc": "2.0", "method": method, "params": params, "id": 1} for params in list_params]
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, headers=headers, json=data)
    return response.json()


def change_log_dict(log_dict):
    """
    Parameters
    ----------
    log_dict: AttributeDict
        Decoded logs.
    Returns
    -------
    args_event: AttributeDict
        Change number basis.
    """
    dictionary = log_dict.copy()
    dictionary['blockHash'] = HexBytes(dictionary['blockHash'])
    dictionary['blockNumber'] = int(dictionary['blockNumber'], 16)
    dictionary['logIndex'] = int(dictionary['logIndex'], 16)
    for i in range(len(dictionary['topics'])):
        dictionary['topics'][i] = HexBytes(dictionary['topics'][i])
    dictionary['transactionHash'] = HexBytes(dictionary['transactionHash'])
    dictionary['transactionIndex'] = int(dictionary['transactionIndex'], 16)
    return AttributeDict(dictionary)


def clean_logs(contract, myevent, log):
    """
    Parameters
    ----------
    contract: web3.eth.contract
        Contract that contains the event.
    myevent: str
        string with event name.
    log: List[AttributeDict]
        List containing raw node response.
    Returns
    -------
    args_event: AttributeDict
        Decoded logs.
    """
    log_dict = AttributeDict({'logs': log})
    eval_string = 'contract.events.{}().processReceipt({})'.format(myevent, log_dict)
    args_event = eval(eval_string)[0]
    return args_event


def get_logs(contract, myevent, hash_create, from_block, to_block, number_batches):
    """
    Get event logs using recursion.
    Parameters
    ----------
    contract: web3.eth.contract
        Contract that contains the event.
    myevent: str
        string with event name.
    hash_create: str
        hash of the event.
    from_block: int
        Starting block.
    to_block: int
        Ending block.
    number_batches: int
        infura returns just 10k logs each call, therefore we need to split time series into batches.
    Returns
    -------
    events_clean: list
        List with all clean logs.
    """

    events_clean = []
    block_list = [int(from_block + i * (to_block - from_block) / number_batches) for i in range(0, number_batches)] + [
        to_block]

    block_list[0] -= 1
    list_params = [[{"address": contract.address,
                     "fromBlock": hex(block_list[i - 1] + 1),
                     "toBlock": hex(block_list[i]),
                     "topics": [hash_create]}] for i in range(1, number_batches + 1)]

    logs = get_rpc_response("eth_getLogs", list_params)
    for j, log in enumerate(logs):
        if list(log.keys())[-1] == "result":
            for event in log['result']:
                log_dict = change_log_dict(event)
                events_clean += [clean_logs(contract, myevent, [log_dict])]
        else:
            events_clean += get_logs(contract, myevent, hash_create, int(list_params[j][0]["fromBlock"], 16),
                                     int(list_params[j][0]["toBlock"], 16), 15)
    return events_clean

In [ ]:
from web3 import Web3, HTTPProvider
shared.init()
from itertools import islice


def connect_to_web3():
    """
    Connect to Web3 server.
    Parameters
    ----------
    Returns
    -------
    res: bool
        True if connection was succeeded, otherwise False
    web3: Web3 Object
    """
    web3 = Web3(HTTPProvider('https://mainnet.infura.io/v3/d6243bb783b44485ad6636b6c3411377'))
    res = web3.isConnected()
    return res, web3


def split_chunks(data, n_elements):
    """
    Splitting the calls to aggregate them properly.
    Parameters
    ----------
    data: np.array
        Containing calls
    n_elements: int
        Calls we want to aggregate
    Returns
    -------
    chunks: list
        Calls in chunks
    """

    chunks = []
    n = len(data)
    
    if n % n_elements != 0:
        n_chunks = int(n/n_elements)+1

    else:
        n_chunks = int(n/n_elements)
            
    for  i in range(n_chunks-1):
        chunk = data[i*n_elements:(i+1)*n_elements]
        chunks.append(chunk)

    chunks.append(data[(n_chunks-1)*n_elements:]) 
    return chunks


def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}


def get_pools(dex, factory): #v2 or sushi
    """
    Parameters
    ----------
    dex : str
        Choose the DEX you want to get the pools from
    factory : web3.eth.contract
        factory get_contracts of dex
    Returns
    -------
    pool_dic: Dict[str, float]
        pool dictionary with the attributes 'address','dex','token0','token1','reserves0','reserves1','creation'
    """

    hash_create = '0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9'
    if dex == 'uniswap_v2':
        from_block = 10008355
        to_block = shared.BLOCKSTUDY
        number_batches = 20
        pools = get_logs(factory, 'PairCreated', hash_create, from_block, to_block, number_batches)
    
    if dex == 'sushiswap':
        from_block = 10822038
        to_block = shared.BLOCKSTUDY
        number_batches = 20
        pools = get_logs(factory, 'PairCreated', hash_create, from_block, to_block, number_batches)

    pool_dic = {}
    tokens = {}
    for pool in pools:
        pool_address = pool.args.pair
        token0 = pool.args.token0
        token1 = pool.args.token1
        tokens.update({token0:None})
        tokens.update({token1:None})
        pool_dic.update({pool_address:{ 'address':pool_address,
                                        'dex':dex,
                                        'token0':token0,
                                        'token1':token1,
                                        'reserves0':None,
                                        'reserves1':None,
                                        'creation':pool.blockNumber}})   

    return pool_dic, tokens


def events_to_json(events):
    json_events = []
    for event in events:
        event_dict = dict(event)
        # event_dict.pop('transactionHash')
        event_dict['transactionHash'] = event_dict['transactionHash'].hex()
        event_dict.pop('blockHash')
        event_dict['args'] = dict(event.args)
        json_events.append(event_dict)
       
    return json_events

In [ ]:
# get_tokens&pools.py

import pandas as pd
import json
shared.init()
def get_token_and_pools(out_path, dex='uniswap_v2'):
    """
    Get tokens and pools from sushiswap or uniswap_v2.
    Parameters
    ----------
    out_path : str
        Path to output directory.
    dex : str
        sushiswap or uniswap_v2 are currently allowed.
    """

    factory = shared.web3.eth.contract(shared.UNISWAP_FACTORY, abi=shared.ABI_FACTORY)
    pool_dic, tokens = get_pools(dex, factory)
    pd.DataFrame(tokens.keys(), columns=["token_address"]).to_csv(f"{out_path}/tokens.csv", index=False)

    with open(f"{out_path}/pool_dict.json", "w") as outfile:
        json.dump(pool_dic, outfile)

    inverted_pool_dict = dict()
    for pool in pool_dic.keys():
        try:
            inverted_pool_dict[pool_dic[pool]['token0']].append(pool_dic[pool])
        except:
            inverted_pool_dict[pool_dic[pool]['token0']] = [pool_dic[pool]]
        try:
            inverted_pool_dict[pool_dic[pool]['token1']].append(pool_dic[pool])
        except:
            inverted_pool_dict[pool_dic[pool]['token1']] = [pool_dic[pool]]

    with open(f"{out_path}/pools_of_token.json", "w") as outfile:
        json.dump(inverted_pool_dict, outfile)

    print('Tokens and Pools downloaded!')


# get_token_and_pools("../data", dex='uniswap_v2')

In [ ]:
import json
shared.init()

def get_pool_events(event_name, hashed_event, pool_address, out_path, start_block, end_block):
    """
    Get pool logs for a given pool and period.
    This function saves the events as a json in out_path.
    Parameters
    ----------
    event_name: str
        Pool event in string format (example: Mint)
    hashed_event: str
        The hashed event. Use obtain_hash_event() if necessary.
    pool_address : str
        Token address.
    out_path : str
        Path to output directory.
    start_block: float
        Starting block.
    end_block: float
        Ending block.
    """

    pool = shared.web3.eth.contract(pool_address, abi=shared.ABI_POOL)
    try:
        events = get_logs(pool, event_name, hashed_event, start_block, end_block, number_batches=10)
    except Exception as err:
        print(f"Exception occured: {err}")
        return

    json_events = events_to_json(events)
    with open(f'{out_path}/{pool_address}.json', 'w+') as f:
        json.dump(json_events, f)
    f.close()

In [ ]:
from bs4 import BeautifulSoup
import requests


def _obtain_tx_creation(token_address):
    """
    Gets token tx creation hash via web scrapping.
    Parameters
    ----------
    token_address: str
        string corresponding to token address
    Returns
    -------
        tx_creation hash if success, "Not found" otherwise.
    """

    tx_hash_creation = "Not found"
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/'
                             '537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    r = requests.get(f'https://etherscan.io/address/{token_address}', headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    soup2 = soup.find_all('div', 'col-md-8')
    for element in soup2:
        if "Creator Txn Hash" in str(element):
            for element2 in str(element).split():
                if 'href' and 'tx' in str(element2):
                    tx_hash_creation = str(element2[10:]).replace('"', '')
    if tx_hash_creation != "Not found":
        return tx_hash_creation
    return "Not found"

In [ ]:
shared.init()
def get_decimal_token(token_address):
    """
    Gets token decimal given a token address.
    Parameters
    ----------
    token_address: str
        String containing token address.
    Returns
    -------
    decimal: int
        Int corresponding to token decimal.
    """

    contract = shared.web3.eth.contract(token_address, abi=shared.ABI)
    decimals = contract.functions.decimals().call()

    return decimals

In [ ]:
import json
import requests
shared.init()

def get_source_code(token_address, out_path):
    """
    Obtains token source code/abi and saves in json format.
    Parameters
    ----------
    token_address: str
        token address in checksum format.
    out_path : str
        Path to output directory.
    """

    source_code_endpoint = "https://api.etherscan.io/api?" \
                           "module=contract" \
                           "&action=getsourcecode" \
                           f"&address={token_address}" \
                           f"&apikey={shared.API_KEY}"
    source_code = json.loads(requests.get(source_code_endpoint).text)['result']

    with open(f"{out_path}/{token_address}.json", "w") as outfile:
        json.dump(source_code, outfile)
    outfile.close()

In [ ]:
import pandas as pd
from web3 import Web3
shared.init()
def get_transfers(token_address, out_path, start_block, end_block, decimal=18):
    """
    Get transfer logs for a given token and period.
    This function saves transfers as a csv in out_path.
    Parameters
    ----------
    token_address : str
        Token address.
    out_path : str
        Path to output directory.
    decimal: float
        Token decimal (usually 18).
    start_block: int
        Starting block.
    end_block: int
        Ending block.
    """

    # Initialise contract objects and get the transactions.
    try:
        contract = shared.web3.eth.contract(Web3.toChecksumAddress(token_address), abi=shared.ABI)
        transfers = get_logs(contract, "Transfer", hash_log, start_block, end_block, number_batches=1)
    except Exception as err:
        print(f"Exception occured: {err}")
        return

    # Save txs in a Dataframe.
    txs = [[transaction['transactionHash'].hex(), transaction["blockNumber"], transaction["args"]['from'],
            transaction["args"]['to'], transaction["args"]['value'] / 10 ** decimal] for transaction in transfers]
    transfers = pd.DataFrame(txs, columns=["transactionHash", "block_number", "from", "to", "value"])
    transfers.to_csv(out_path + "/" + token_address + ".csv", index=False)
    return


hash_log = obtain_hash_event('Transfer(address,address,uint256)')
get_transfers('0xa150Db9b1Fa65b44799d4dD949D922c0a33Ee606', './', 11000000, 11025824) # example